# NLP NER - Rujuta Gandhi

You have been provided with a pickle file, containing the 100 news articles about Caterpillar.  Identify what companies are mentioned most frequently in the news along with Caterpillar. 

- pd.read_pickle(directory+'news_cat.pkl')
- Discard non-English results
- Identify what companies are mentioned most frequently along with Caterpillar (in both title and the body of the article)
- Show a table or chart with your top-20 companies (sorted in the descending order)

### Import Library, File, English Only Results

In [1]:
import pyforest
import pandas as pd

In [2]:
import spacy 
from spacy import displacy
nltk.download('stopwords')

<IPython.core.display.Javascript object>

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gandh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#SpaCy 2.x brough significant speed and accuracy improvements
spacy.__version__

'2.2.3'

In [4]:
df = pd.read_pickle(r"C:\Users\gandh\Google Drive\UChicago\11_Quarter 10\Assignments\Assignment 5\news_cat.pkl")

In [5]:
df.head()

,crawled,language,text,title
0,2018-01-30T23:03:51.004+02:00,english,by Abhishek K Global Telehandler Market 2023 D...,Global Telehandler Market 2023 Demand by Segme...
1,2018-01-30T23:06:46.024+02:00,english,favorite this post 2014 Caterpillar 314E LCR h...,2014 Caterpillar 314E LCR
2,2018-01-30T23:18:35.023+02:00,english,By: MAX NISEN The Amazon health care threat ha...,"Amazon, Berkshire, JPMorgan health announcemen..."
3,2018-01-30T23:20:54.012+02:00,english,QR Code Link to This Post MONTHLY PUBLIC AUCTI...,2005 Caterpillar CB534D Tandem Vibratory Rolle...
4,2018-01-30T23:28:30.000+02:00,english,QR Code Link to This Post 2007 CATERPILLAR D4G...,2007 CATERPILLAR D4G LGP CAB SCREEN/SWEEPS - O...


In [6]:
df.shape

(100, 4)

In [7]:
df.language.value_counts()

english    100
Name: language, dtype: int64

#### The file already has English only results

df = df[df.language=='english']
df.head()

In [8]:
df = df.drop(columns=['crawled','language'])

#### Load Spacy Model


In [9]:
# Load SpaCy model
nlp = spacy.load('en_core_web_sm')

#### Identify what companies are mentioned most frequently along with Caterpillar (in both title and the body of the article)
- Get text and title in the same columns
- Keep rows only if they contain some version of Caterpillar

In [10]:
text_updated = pd.DataFrame(df['text'][df['text'].str.contains('Cat|CAT')==True]).rename(columns={"text": "title_text"}).reset_index(drop=True)
text_updated

,title_text
0,by Abhishek K Global Telehandler Market 2023 D...
1,favorite this post 2014 Caterpillar 314E LCR h...
2,QR Code Link to This Post MONTHLY PUBLIC AUCTI...
3,QR Code Link to This Post 2007 CATERPILLAR D4G...
4,Elite Wealth Management Inc. Acquires Shares o...
...,...
92,What to Expect From Caterpillar Inc. in 2018 J...
93,transmission: other QR Code Link to This Post ...
94,One year after Caterpillar's headquarters anno...
95,"1,613 Shares in Caterpillar Inc. (NYSE:CAT) Pu..."


In [11]:
title_updated = pd.DataFrame(df['title'][df['title'].str.contains('Cat|CAT')==True]).rename(columns={"title": "title_text"}).reset_index(drop=True)
title_updated.shape

(53, 1)

In [12]:
frame=[title_updated,text_updated]
text_title = pd.concat(frame).reset_index(drop=True)
text_title.head()

,title_text
0,2014 Caterpillar 314E LCR
1,2005 Caterpillar CB534D Tandem Vibratory Rolle...
2,2007 CATERPILLAR D4G LGP CAB SCREEN/SWEEPS - O...
3,Elite Wealth Management Inc. Acquires Shares o...
4,Caterpillar T40D LP Forklift Tow motor Triple ...


In [13]:
text_title = nlp(text_title['title_text'].str.cat(sep=' | ').replace('\n', '').replace('\r',''))

In [14]:
text_title

2014 Caterpillar 314E LCR | 2005 Caterpillar CB534D Tandem Vibratory Roller (downtown / civic / van ness) | 2007 CATERPILLAR D4G LGP CAB SCREEN/SWEEPS - ORIGINAL LOW HOURS (SWEENY, TX) $65999 | Elite Wealth Management Inc. Acquires Shares of 2,191 Caterpillar Inc. (CAT) | Caterpillar T40D LP Forklift Tow motor Triple Mast (Claymont) $4499 | Somewhat Favorable News Coverage Somewhat Unlikely to Impact Caterpillar (CAT) Share Price | CATERPILLAR D8 IN SEARCH OF (MICHIGAN) $1 | Caterpillar Marine announces open order board on new Cat® C7 | Security National Trust Co. Acquires 495 Shares of Caterpillar Inc. (CAT) | One year later, Peoria progresses after Caterpillar headquarters announcement | Caterpillar 938G Loader $35000 | Global Diesel Power Engine Market 2018-2022 with Caterpillar, Wartsila, Cummins, Rolls-Royce, and Mitsubishi Heavy Industries Dominating - ResearchAndMarkets.com | Caterpillar : One year later, Peoria progresses after Caterpillar headquarters announcement | Caterpilla

#### Get Organization list with NER
- Remove other label types

In [15]:
entities = []
labels = []

for ent in text_title.ents:
    entities.append(ent)
    labels.append(ent.label_)
    
tt_entities = pd.DataFrame({'Entities':entities,'Labels':labels})

tt_entities.head(20)

,Entities,Labels
0,(2014),CARDINAL
1,(314E),CARDINAL
2,(LCR),ORG
3,(2005),DATE
4,(2007),DATE
5,(SWEENY),ORG
6,(TX),ORG
7,(65999),DATE
8,"(2,191)",CARDINAL
9,"(Caterpillar, Inc.)",ORG


In [16]:
filt = tt_entities[(tt_entities['Labels']=='ORG')].reset_index(drop=True).drop('Labels',axis=1)

In [17]:
filt

,Entities
0,(LCR)
1,(SWEENY)
2,(TX)
3,"(Caterpillar, Inc.)"
4,(CAT)
...,...
2138,(HoldingsChannel.com)
2139,"(Caterpillar, Inc.)"
2140,(NYSE)
2141,(Caterpillar)


#### Show a table or chart with your top-20 companies (sorted in the descending order)
- drop any occurrences of Caterpillar in any form

In [18]:
filt = filt.astype('str')

In [19]:
final = pd.DataFrame(filt['Entities'][filt['Entities'].str.contains('Cat|CAT')==False]).reset_index(drop=True)
entity_counts = pd.DataFrame(final.groupby('Entities')['Entities'].count().sort_values(ascending = False))
entity_counts.head(20)

,Entities
Entities,
NYSE,63
SEC,23
Amazon,22
Vista,22
Resource Industries,21
Energy & Transportation,19
EPS,18
Construction Industries,18
Company,18
